# Importando as bibliotecas

In [2]:
import pandas as pd
import numpy as np
import datetime
from plotly.offline import iplot
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import os
import os.path as path

#dataframe do o arquivo .xlsx
cd = pd.read_excel("STG_FNT_ITT.xlsx")

#pd.set_option('display.max_rows', None)

In [3]:
#Cria as pastas necessárias
pasta ="./relatorio/pags"

access_rights = 0o777

try:
    os.mkdir('relatorio', access_rights)
except OSError:
    print ("\tAVISO: A criação da pasta (relatorio) falhou ou a pasta já existe")
else:
    print ("\tPasta (relatorio) criada com sucesso!")

try:
    os.mkdir('relatorio/pags', access_rights)
except OSError:
    print ("\tAVISO: A criação da pasta (pags) falhou ou a pasta já existe")
else:
    print ("\tPasta (pags) criada com sucesso!")

	Pasta (relatorio) criada com sucesso!
	Pasta (pags) criada com sucesso!


# Valida os tipos das colunas

In [19]:
#Trata as datas
cd.DAT_INC_DBO = pd.to_datetime(cd.DAT_INC_DBO, format='%Y-%m-%d', errors='coerce')

#Verifica as linhas das colunas o tipo desejado
tipo_ID_STG_FNT_ITT = len([x for x in range(len(cd)) if cd.ID_STG_FNT_ITT[x].dtype == np.int64])
tipo_NUM_CNPJ = len([x for x in range(len(cd)) if cd.NUM_CNPJ[x].dtype == np.int64])
tipo_NUM_CMP_CNPJ = len([x for x in range(len(cd)) if cd.NUM_CMP_CNPJ[x].dtype == np.int64])
tipo_NOM_COM = len([x for x in range(len(cd)) if type(cd.NOM_COM[x]) == str])
tipo_NOM_RAZ_SCL = len([x for x in range(len(cd)) if type(cd.NOM_RAZ_SCL[x]) == str])
tipo_DAT_INC_DBO = len([x for x in range(len(cd)) if isinstance(cd.DAT_INC_DBO[x], datetime.date)])

#Define variáveis para criação do dataframe
tipos = [tipo_ID_STG_FNT_ITT, tipo_NUM_CNPJ, tipo_NUM_CMP_CNPJ, tipo_NOM_COM,
         tipo_NOM_RAZ_SCL, tipo_DAT_INC_DBO]
tipos = [tipo * 100 / len(cd) for tipo in tipos]

#Cria o dataframe
tipo_review = pd.DataFrame({'Porcentagem': tipos},
                           index=cd.columns)

#Plota em formato gráfico e exporta para HTML
data = [go.Bar(x=tipos,
               y=cd.columns,
               orientation = 'h',
               marker=dict(color='#3749E9')
              )]

fig = go.Figure(data=data)
fig.update_yaxes(showline=True, linewidth=1, linecolor='#717171')
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#D9D9DE')
fig.update_layout(dict(plot_bgcolor = 'rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)'))
iplot(fig)
fig.write_html(pasta + "/fnt_tipo.html")

# Define as colunas com dados sensíveis

In [20]:
#Cria um dataframe definido as colunas com dados sensíveis
dado_sensivel = ['Sim', 'Sim', 'Sim', 'Sim', 'Sim', 'Não']
sensibilidade_review = pd.DataFrame({'Dado Sensível': dado_sensivel},
                                      index=cd.columns)
sensibilidade_review = sensibilidade_review.rename_axis('Colunas', axis='columns')

In [21]:
#Plota em formato tabela e exporta para HTML
cores = []
for x in range (len (sensibilidade_review)):
    if x % 2 == 0:
        cores.append('#DEDEDE')
    else:
        cores.append('#ECECEC')

fig = go.Figure(data=[go.Table(
    header=dict(values=['<b>Colunas</b>', '<b>Dado Sensível</b>'],
                font=dict(color='#707070', size=14),
                height=30,
                line_color = '#707070',
                fill_color='#ECECEC',
                align='left'),
    cells=dict(values=[sensibilidade_review.index,
                       dado_sensivel],
               font=dict(color='#707070', size=14),
               height=30,
               line_color = '#707070',
               fill_color=[cores*2],
               align=['left', 'right', 'right', 'right']))
])

fig.update_layout(xaxis={'categoryorder':'total ascending'})

iplot(fig)
fig.write_html(pasta + "/fnt_tab_sensibilidade.html")

# Verifica a completude da tabela

In [22]:
analisar = cd.notnull().count() # Contando quantos não são nulos.

total_linhas = cd.shape[0] # Pegando o total de linhas.

# Colocando em um dicionário os nomes das colunas e seus respectivos totais de não nulos.
# Também transformando os totais em porcentagem.
situacao = {x[0] : (x[1] * 100 / total_linhas) for x in analisar.items()}

# Criando um DataFrame com as informações recolhidas.
situacao_plot = pd.DataFrame({'COMPLETUDE (%)' : [x for x in situacao.values()]},
                             index = [x for x in situacao.keys()])

In [23]:
#Plota em gráfico de barras horizontal
data = go.Bar(x = [x for x in situacao.values()], 
              y = [x for x in situacao.keys()], 
              orientation = 'h', 
              marker = {'color' : '#3749E9'})

layout = go.Layout(title = '', 
                   yaxis = {'title': ''}, 
                   xaxis = {'title': 'Porcentagem da Validação (%)'})

fig = go.Figure(data = data, layout = layout)
fig.update_yaxes(showline = True, linewidth = 1, linecolor = '#717171')
fig.update_xaxes(showgrid = True, gridwidth = 1, gridcolor = '#D9D9DE')
fig.update_layout({'plot_bgcolor': '#FFFFFF', 'paper_bgcolor': '#FFFFFF'})
iplot(fig)
fig.write_html(pasta + "/fnt_preenchimento.html")

In [24]:
#Separa por grupos os valores diferentes encontrados nas colunas de datas
datas_inc = cd[['DAT_INC_DBO', 'ID_STG_FNT_ITT']]
datas_inc = datas_inc.groupby('DAT_INC_DBO').count().sort_values(by = 'ID_STG_FNT_ITT', ascending = False)

#Analisa os valores únicos encontrados e calcula a porcentagem de seu tamanho com o número de linhas da tabela
conta = 0
porcentagem = []
for x in datas_inc.ID_STG_FNT_ITT:
    conta = x * 100 / len(cd)
    porcentagem.append (conta)

#Acrescenta o resultado ao dataframe
datas_inc['Porcentagem'] = porcentagem
datas_inc = datas_inc.rename_axis('Datas Encontradas', axis='columns').drop('ID_STG_FNT_ITT', 1)


#Concatena e cria uma tabela com todos os resultados
dados = [datas_inc]
datas_encontradas = pd.concat(dados)
datas_encontradas.index.name = None
datas_encontradas

Datas Encontradas,Porcentagem
2019-12-19 13:43:41.914,100.0


In [25]:
#Plota em formato tabela e exporta para HTML
cores = []
for x in range (len (datas_encontradas)):
    if x % 2 == 0:
        cores.append('#DEDEDE')
    else:
        cores.append('#ECECEC')
        
fig = go.Figure(data=[go.Table(
    header=dict(values=['<b>Datas Encontradas</b>','<b>Porcentagem</b>'],
                font=dict(color='#707070', size=14),
                height=30,
                line_color = '#707070',
                fill_color='#ECECEC',
                align='left'),
    cells=dict(values=[datas_encontradas.index,
                       datas_encontradas.Porcentagem.map('{:,.0f}%'.format)],
               font=dict(color='#707070', size=14),
               height=30,
               line_color = '#707070',
               fill_color=[cores*2],
               align=['left', 'right', 'right', 'right']))
])

fig.update_layout(xaxis={'categoryorder':'total ascending'})

iplot(fig)
fig.write_html(pasta + "/fnt_tab_datas.html")

# Analisa os CNPJs válidos desconsiderado linhas nulas, cnpjs inválidos e nomes idênticos

In [26]:
#Drepeza as linhas vazias
df = cd[cd['ID_STG_FNT_ITT'].notna()]
df = df[df['NUM_CNPJ'].notna()]
df = df[df['NUM_CMP_CNPJ'].notna()]
df = df[df['NOM_COM'].notna()]
df = df[df['NOM_RAZ_SCL'].notna()]
df = df[df['DAT_INC_DBO'].notna()]

In [27]:
#Agrupa por nomes idênticos
analise = df.groupby('NOM_RAZ_SCL').count().sort_values(by = 'NUM_CNPJ', ascending = False)
analise

#Depreza os nomes idênticos
nome_repetido = analise[analise['ID_STG_FNT_ITT'] > 1]
lista = nome_repetido.index

#Adiciona a listas os CNPJs por nomes idênticos ou únicos
cnpj_inv = []
cnpj_c_inv = []
cnpj_val = []
cnpj_c_val = []
for x in range (len (df)):
    if df.NOM_RAZ_SCL[x] in lista:
        cnpj_inv.append (df.NUM_CNPJ[x])
        cnpj_c_inv.append (df.NUM_CMP_CNPJ[x])
    else:    
        cnpj_val.append (df.NUM_CNPJ[x])
        cnpj_c_val.append (df.NUM_CMP_CNPJ[x])

In [28]:
#Analisa apenas os CNPJs cuja a linha da coluna NUM_CNPJ possui valor único
cnpjList_val = []
for c in range (len(cnpj_val)):
    cnpj = str(cnpj_val[c]) + str(cnpj_c_val[c])
    cnpjList_val.append(cnpj)

#Verifica o tamanho do CNPJ e adiciona a listas de validos ou inválidos, com mais ou com menos de 14 caracteres
cnpj_valido_val, cnpj_valido, cnpj_nao_valido = [], [], []
cnpj_mais, cnpj_menos = [], []
for c in range (len(cnpjList_val)):
    charCNPJ = len(cnpjList_val[c])
    if charCNPJ == 14:
        cnpj_valido_val.append(cnpjList_val[c])
    else:
        cnpj_nao_valido.append(cnpjList_val[c])
    if charCNPJ > 14:
        cnpj_mais.append(cnpjList_val[c])
    if charCNPJ < 14:
        cnpj_menos.append(cnpjList_val[c])

#Analisa apenas os CNPJs cuja a linha da coluna NUM_CNPJ possui valores idênticos
cnpjList_inv = []
for c in range (len(cnpj_inv)):
    cnpj = str(cnpj_inv[c]) + str(cnpj_c_inv[c])
    cnpjList_inv.append(cnpj)

#Verifica o tamanho do CNPJ e adiciona a listas de validos ou inválidos, com mais ou com menos de 14 caracteres
for c in range (len(cnpjList_inv)):
    charCNPJ = len(cnpjList_inv[c])
    if charCNPJ == 14:
        cnpj_valido.append(cnpjList_inv[c])
    else:
        cnpj_nao_valido.append(cnpjList_inv[c])
    if charCNPJ > 14:
        cnpj_mais.append(cnpjList_inv[c])
    if charCNPJ < 14:
        cnpj_menos.append(cnpjList_inv[c])

#len(cnpjList_val), len(cnpjList_inv), len(cnpj_valido_val), len(cnpj_valido), len(cnpj_nao_valido), len(cnpj_mais), len(cnpj_menos)

# Gráfico dos CNPJs Únicos

In [30]:
#Plota em gráfico de setores
cnpjs_unicos = len(np.unique(cnpj_valido_val))
cnpjs_unicos = cnpjs_unicos + len(np.unique(cnpj_valido))
cnpjs_identicos = len(cnpj_valido_val) + len(cnpj_valido) - cnpjs_unicos

valores_unicos = cnpjs_unicos*100/(len(cnpj_valido_val) + len(cnpj_valido))
valores_identicos = 100 - valores_unicos

labels = ['CNJPs Validados Únicos', 'CNJPs Validados Idênticos']
colors = ['#cfe509', '#3749e9', '#112244']
sizes = [valores_unicos, valores_identicos]
explode = (0, 0.05)

fig = go.Figure(data=[go.Pie(labels=labels, values=sizes)])
fig.update_traces(marker=dict(colors=colors, line=dict(color='#000000', width=0)))
fig.show()
fig.write_html(pasta + "/fnt_cnpjs_unicos.html")

# Gráfico de todos os CNPJs válidos desconsiderado linhas nulas, cnpjs inválidos e nomes idênticos

In [14]:
#Plota em gráfico de setores
cnpjs_unicos = np.unique(cnpj_valido_val)
cnpjs_unicos = len(cnpjs_unicos)
cnpjs_identicos = len(cnpj_valido_val) - cnpjs_unicos

#CNPJs válidos
op_vld = cnpjs_unicos
vld = (op_vld*100) / len(df)

#CNPJs descartados, mas válidos
op_inv = len(cnpj_valido) + cnpjs_identicos
inv = (op_inv*100) / len(df)

#CNPJs não válidos
op_nvld = len(cnpj_nao_valido)
nvld = (op_nvld*100) / len(df)

labels = 'Válidos', 'Invalidados', 'Não válidos'
colors = ['#cfe509', '#3749e9', '#112244']
sizes = [vld, inv, nvld]
explode = (0, 0.05, 0.05)

fig = go.Figure(data=[go.Pie(labels=labels, values=sizes)])
fig.update_traces(marker=dict(colors=colors, line=dict(color='#000000', width=0)))
fig.show()
fig.write_html(pasta + "/fnt_cnpjs_validos.html")

# Gráfico de todos os CNPJs por número de carácteres

In [15]:
#Plota em gráfico de setores
# +14 Char
op_mais14 = len(cnpj_mais)
mais_14_char = (op_mais14*100) / len (df)

# -14 Char
op_menos14 = len(cnpj_menos)
menos_14_char = (op_menos14*100) / len (df)

labels = '+14 Caracteres', '-14 Caracteres', '14 Caracteres (Correto)'
colors = ['#cfe509', '#3749e9', '#112244']
sizes = [mais_14_char, menos_14_char, vld+inv]
explode = (0.03, 0.03, 0.03)

fig = go.Figure(data=[go.Pie(labels=labels, values=sizes)])
fig.update_traces(marker=dict(colors=colors, line=dict(color='#000000', width=0)))
fig.show()
fig.write_html(pasta + "/fnt_cnpjs_caracteres.html")

# Gráfico de Nomes Únicos

In [16]:
#Plota em gráfico de setores
nome_identico = analise[analise['ID_STG_FNT_ITT'] > 1]
nome_identico = nome_identico.ID_STG_FNT_ITT.sum()*100/len(df)
nome_unico = 100 - nome_identico

labels = ['Nomes Únicos', 'Nomes Idênticos']
colors = ['#cfe509', '#3749e9', '#112244']
sizes = [nome_unico, nome_identico]
explode = (0, 0.05)

fig = go.Figure(data=[go.Pie(labels=labels, values=sizes)])
fig.update_traces(marker=dict(colors=colors, line=dict(color='#000000', width=0)))
fig.show()
fig.write_html(pasta + "/fnt_nomes_unicos.html")